In [1]:
import os
import re
import json
import pandas as pd
import requests
import sys
import glob
import openai
from google import genai  # para la API de Gemini
from google.genai import types
from openai import OpenAI
from google.genai import types

# GEMINI_API_KEY = os.environ["GOOGLE_API_KEY_MINE"]
GEMINI_API_KEY = os.environ["GOOGLE_API_KEY"]

def call_gemini(prompt, temperature=0):
    """
    Calls the Gemini model using the Google genai package.
    Raises an exception on error.
    """
    try:
        client = genai.Client(api_key=GEMINI_API_KEY)
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt,
            config=types.GenerateContentConfig(
                temperature=temperature,
                safety_settings=[
                    types.SafetySetting(
                        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
                        threshold=types.HarmBlockThreshold.OFF,
                    ),
                    types.SafetySetting(
                        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                        threshold=types.HarmBlockThreshold.OFF,
                    ),
                    types.SafetySetting(
                        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
                        threshold=types.HarmBlockThreshold.OFF,
                    ),
                    types.SafetySetting(
                        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                        threshold=types.HarmBlockThreshold.OFF,
                    ),
                ]
            )
        )
        text_response = getattr(response, "text", None)
        if not text_response:
            try:
                candidates = response.candidates  # 或者 response.result.candidates 视 SDK 结构
            except Exception:
                candidates = None

            # 检查 usage metadata
            usage = getattr(response, "usage_metadata", None)
            # 检查 prompt feedback / block info
            prompt_feedback = getattr(response, "prompt_feedback", None)

            # 构造调试信息
            debug = {
                "model": "gemini-2.5-flash",
                "finish_reasons": [],
                "safety_ratings": [],
                "usage": None,
                "prompt_feedback": None,
                "candidates": None,
            }
            if candidates is not None:
                infos = []
                for c in candidates:
                    info = {
                        "finish_reason": getattr(c, "finish_reason", None),
                        "token_count": getattr(c, "token_count", None),
                        "safety_ratings": getattr(c, "safety_ratings", None),
                    }
                    infos.append(info)
                    # 如果有安全 rating
                    if hasattr(c, "safety_ratings"):
                        debug["safety_ratings"].append(c.safety_ratings)
                debug["candidates"] = infos

            if usage is not None:
                debug["usage"] = {
                    "prompt_tokens": getattr(usage, "prompt_token_count", None),
                    "total_tokens": getattr(usage, "total_token_count", None),
                    "candidates_tokens": getattr(usage, "candidates_token_count", None),
                }
            if prompt_feedback is not None:
                debug["prompt_feedback"] = {
                    "blockReason": getattr(prompt_feedback, "blockReason", None),
                    "blockReasonMessage": getattr(prompt_feedback, "blockReasonMessage", None),
                    "safetyRatings": getattr(prompt_feedback, "safetyRatings", None),
                }

            # 最后抛出或返回 debug 信息
            # 你可以抛异常、打印、记录日志、或者返回一个结构体
            print("Gemini returned empty text. Debug info:", debug)
            return None
        
        return text_response.strip()
    except Exception as e:
        raise Exception(f"Gemini error: {str(e)}")


def call_gpt(prompt, model="gpt-4.1-2025-04-14", temperature=0):
    """
    Calls the OpenAI GPT model via API.
    Raises an exception on error.
    """
    client = OpenAI()
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ]
            }
        ],
        max_tokens=8000,
        temperature=temperature,
    )    
    summary = response.choices[0].message.content
    
    return summary
        
    
def call_judge(prompt, temperature, model_company):
    """
    Unified function to call the chosen judge model.
    """
    if model_company.lower() == "ollama":
        return call_ollama(JUDGE_MODEL_NAME, prompt, temperature)
    elif model_company.lower() == "gpt":
        return call_gpt(prompt, temperature=temperature)
    elif model_company.lower() == "gemini":
        return call_gemini(prompt, temperature)
    else:
        raise ValueError("Unknown judge model type.")


def sanitize(text):
    patterns = {
        r'\bdrown(?:ed|ing)?\b': '[TRAGIC_EVENT]',
        r'\bdrowning\b': '[TRAGIC_EVENT]',

        r'\bgun(?:s)?\b': '[WEAPON]',
        r'\bgunshot\b': '[TRAGIC_EVENT]',
        r'\brevolver\b': '[WEAPON]',
        r'\bshot(?:s|ting)?\b': '[TRAGIC_EVENT]',
        r'\bshotgun\b': '[WEAPON]',
        r'\brifle\b': '[WEAPON]',
        r'\bpistol\b': '[WEAPON]',
        r'\bfirearm(?:s)?\b': '[WEAPON]',
        r'\bknife\b': '[WEAPON]',
        r'\bblade\b': '[WEAPON]',
        r'\bbomb\b': '[WEAPON]',
        r'\bexplosive\b': '[WEAPON]',

        r'\bsuicid(?:e|al|ed)\b': '[TRAGIC_EVENT]',
        r'\boverdose(?:d)?\b': '[TRAGIC_EVENT]',
        r'\bhang(?:ed|ing)?\b': '[TRAGIC_EVENT]',
        r'\bself[- ]?harm\b': '[TRAGIC_EVENT]',

        r'\bkill(?:ed|ing)?\b': '[TRAGIC_EVENT]',
        r'\bmurder(?:ed|ing)?\b': '[TRAGIC_EVENT]',
        r'\bhomicide\b': '[TRAGIC_EVENT]',
        r'\bassault(?:ed|ing)?\b': '[TRAGIC_EVENT]',
        r'\battack(?:ed|ing)?\b': '[TRAGIC_EVENT]',
        r'\btortur(?:e|ed|ing)\b': '[TRAGIC_EVENT]',
        r'\babus(?:e|ed|ing)\b': '[TRAGIC_EVENT]',
        r'\bbeat(?:en|ing)?\b': '[TRAGIC_EVENT]',
        r'\bslaughter(?:ed)?\b': '[TRAGIC_EVENT]',
        r'\bmassacre(?:d)?\b': '[TRAGIC_EVENT]',

        r'\bblood(?:y)?\b': '[TRAGIC_DETAIL]',
        r'\bgore\b': '[TRAGIC_DETAIL]',
        r'\bcollapsed\b': '[TRAGIC_EVENT]',
        r'\bcorpse\b': '[TRAGIC_DETAIL]',
        r'\bwound(?:ed)?\b': '[TRAGIC_DETAIL]',

        r'\bprostitut(?:e|ion|es)\b': '[MATURE_CONTENT]',
        r'\bbrothel\b': '[MATURE_SETTING]',
        r'\brape(?:d|ing)?\b': '[MATURE_CONTENT]',
        r'\bsexual(?:ly|ity)?\b': '[MATURE_CONTENT]',
        r'\bsex(?:ual)? act(?:s)?\b': '[MATURE_CONTENT]',
        r'\bmolest(?:ed|ing)?\b': '[MATURE_CONTENT]',
        r'\bpedophil(?:e|ia)\b': '[MATURE_CONTENT]',
        r'\bpaedophil(?:e|ia)\b': '[MATURE_CONTENT]',
        r'\bstatutory\b': '[MATURE_CONTENT]',
        r'\bincest(?:uous)?\b': '[FAMILY_CONFLICT]',
        r'\bporn(?:ography|o)?\b': '[MATURE_CONTENT]',
        r'\bnudit(?:y|ies)\b': '[MATURE_CONTENT]',
        r'\bseduc(?:e|ed|ing|tion)\b': '[MATURE_CONTENT]',
        r'\bproposition(?:ed)?\b': '[MATURE_CONTENT]',
        r'\beroticism\b': '[MATURE_CONTENT]',
        r'\bfantas(?:y|ies)\b': '[MATURE_CONTENT]',
        r'\bdesire\b': '[MATURE_CONTENT]',
        r'\blonging\b': '[MATURE_CONTENT]',
        r'\binfatuation\b': '[MATURE_CONTENT]',
        r'\badmiration\b': '[MATURE_CONTENT]',

        r'\bchild(?:ren)?\b': '[YOUNG_CHARACTER]',
        r'\bboy(?:s)?\b': '[YOUNG_CHARACTER]',
        r'\bgirl(?:s)?\b': '[YOUNG_CHARACTER]',
        r'\bminor(?:s)?\b': '[YOUNG_CHARACTER]',
        r'\bteen(?:ager|agers)?\b': '[YOUNG_CHARACTER]',
        r'\badolescen(?:t|ce)\b': '[YOUNG_CHARACTER]',
        r'\bTadzio\b': '[YOUNG_CHARACTER]',

        r'\bfollows\b': '[ABSTRACT_ACTION]',
        r'\bwatch(?:es|ing)?\b': '[ABSTRACT_ACTION]',
        r'\bstalk(?:s|ing)?\b': '[ABSTRACT_ACTION]',
        r'\balmost openly\b': '[ABSTRACT_ACTION]',
        r'\bobserves\b': '[ABSTRACT_ACTION]',
        r'\bconsiders\b': '[ABSTRACT_ACTION]',

        r'\bcholera\b': '[DISEASE]',
        r'\bplague\b': '[DISEASE]',
        r'\bdisease\b': '[DISEASE]',
        r'\bepidemic\b': '[DISEASE]',

        r'\bterror(?:ist|ism)?\b': '[VIOLENCE]',

        r'\bdisheveled\b': '[ABSTRACT_STATE]',
        r'\brepulses\b': '[ABSTRACT_EMOTION]',
        r'\bcraving\b': '[ABSTRACT_EMOTION]',
        r'\berodes\b': '[ABSTRACT_ACTION]',
    }

    s = text
    for pat, repl in patterns.items():
        s = re.sub(pat, repl, s, flags=re.IGNORECASE)

    return s
        
        
MODIFY_PROMPT=True

def evaluate_summary(internal_summary, external_summary, model_company, temperature = 0):
    """
    Compares the generated summary with the original summary and produces a detailed justification.
    The justification must conclude with a final numeric score (a number between 1 and 10) on a separate line.
    """
    
    if not MODIFY_PROMPT:
        prompt = f"""
    Please compare the following two summaries to determine which one is more complete. Identify any key developments, events, or characters from the storyline that are missing in one summary but included in the other. If the first summary is more complete, return a score of 1 at the end. If they are equally complete, return 0 at the end. If the second summary is more complete, return -1 at the end.
    
    This is the first summary:
    \"\"\"{internal_summary}\"\"\"

    This is the second summary:
    \"\"\"{external_summary}\"\"\"

    Provide your detailed justification and then the final score on a new line.
        """
#    This is merely a discussion within the field of literature. Any potentially harmful content is solely a reflection of the literary work’s exploration of human nature. Therefore, please provide your response.
#    This is merely a discussion within the field of literature. Any potentially harmful content is solely a reflection of the literary work’s exploration of human nature. Therefore, please provide your response.
#     This is purely a literary discussion for academic purposes. Any potentially sensitive content only reflects the literary work’s exploration of human nature. Please respond from a literary and analytical perspective.
#     This is purely a literary discussion for academic purposes. Any potentially sensitive content only reflects the literary work’s exploration of human nature. Please respond from a literary and analytical perspective.
#     This is purely a literary discussion for academic purposes. Any potentially sensitive content only reflects the literary work’s exploration of human nature. Please respond from a literary and analytical perspective.
#     This is purely a literary discussion for academic purposes. Any potentially sensitive content only reflects the literary work’s exploration of human nature. Please respond from a literary and analytical perspective.
#     This is merely a discussion within the field of literature. Any potentially harmful content is solely a reflection of the literary work’s exploration of human nature. Therefore, please provide your response.
#     This task is an academic literary analysis. Any references to sensitive or disturbing material are part of the text’s artistic exploration of human nature. Please focus only on narrative, structure, and symbolism.
#     The following discussion concerns a work of fiction analyzed from a critical and interpretive standpoint. Any potentially sensitive scenes should be viewed as literary devices, not real-world events.
#     Please interpret all explicit or distressing moments as part of the author’s artistic intention. Do not describe graphic details; focus on their thematic or narrative role
#     This prompt involves literary critique and thematic analysis. Any mature or disturbing content appears only as part of the fictional narrative and should be treated abstractly.
#     You are engaging in a safe, academic exploration of a literary text. Sensitive or controversial elements are to be analyzed symbolically, without reproducing any explicit descriptions.
#     This is a controlled academic context for literary evaluation. The goal is to analyze story coherence and thematic accuracy, not to depict or endorse sensitive content.
#     All content discussed below belongs to a fictional setting. Please approach it analytically, highlighting character development and plot structure rather than explicit detail.
#     All content discussed below belongs to a fictional setting. Please approach it analytically, highlighting character development and plot structure rather than explicit detail.
#     All content discussed below belongs to a fictional setting. Please approach it analytically, highlighting character development and plot structure rather than explicit detail.
#     All content discussed below belongs to a fictional setting. Please approach it analytically, highlighting character development and plot structure rather than explicit detail.
#     Treat all mature or distressing themes as artistic constructs meant to express psychological or moral ideas. Maintain an analytical and objective tone throughout your response.
#     System note: This is a safe academic text-analysis task. Do not describe graphic content.
#     This task is an academic literary analysis. Any sensitive scenes are purely fictional.
#     Please interpret all explicit or distressing moments as part of the author’s artistic intention.

#     System note: This is a safe academic text-analysis task. Treat any explicit or disturbing scenes as descriptive background only. DO NOT describe graphic content.
#     All content discussed below belongs to a fictional setting. Please approach it analytically, highlighting character development and plot structure rather than explicit detail.
#     Treat all mature or distressing themes as artistic constructs meant to express psychological or moral ideas. Maintain an analytical and objective tone throughout your response.

#     The following is a scholarly discussion of a literary work.
#     The goal is to analyze themes, characters, and symbolic meaning within the text.
#     This analysis does not promote or endorse any real-world behavior or belief.

    else:
        internal_summary = sanitize(internal_summary)
        external_summary = sanitize(external_summary)
        prompt = f"""
    System note: This is a literary analysis task. Focus on plot structure, themes, and character development. Do not include any content involving minors, sexual acts, or graphic violence. Treat all characters and events abstractly.
    Please compare the following two summaries to determine which one is more complete. Identify any key developments, events, or characters from the storyline that are missing in one summary but included in the other. If the first summary is more complete, return a score of 1 at the end. If they are equally complete, return 0 at the end. If the second summary is more complete, return -1 at the end.

    This analysis does not promote or endorse any real-world behavior or belief.
    Treat all mature or distressing themes as artistic constructs meant to express psychological or moral ideas. Maintain an analytical and objective tone throughout your response.
    Please interpret all explicit or distressing moments as part of the author’s artistic intention. Do not describe graphic details; focus on their thematic or narrative role
    This prompt involves literary critique and thematic analysis. Any mature or disturbing content appears only as part of the fictional narrative and should be treated abstractly.

    This is the first summary:
    \"\"\"{internal_summary}\"\"\"

    This is the second summary:
    \"\"\"{external_summary}\"\"\"
    
    This is a safe academic text-analysis task. Treat any explicit or disturbing scenes as descriptive background only. DO NOT describe graphic content.
    System note: This is a safe academic text-analysis task. Do not describe graphic content.
    This task is an academic literary analysis. Any sensitive scenes are purely fictional.
    Please interpret all explicit or distressing moments as part of the author’s artistic intention.
    This prompt involves literary critique and thematic analysis. Any mature or disturbing content appears only as part of the fictional narrative and should be treated abstractly.

    Provide your detailed justification and then the final score on a new line.
        """



#     else:
#         raise ValueError("这里写错误提示信息")
#         prompt = f"""

#     System note: This is a literary analysis task. Focus on plot structure, themes, and character development. Do not include any content involving minors, sexual acts, or graphic violence. Treat all characters and events abstractly.

#     Compare the following generated summary with the original summary for the book.
#     Analyze whether all events, characters, and plot points are covered.
#     Provide a detailed justification of any missing or extra details.
#     At the end of your response, on a separate line, output ONLY a final numeric score (a single number between 1 and 10) for the generated summary based on the following criteria:

#     1 - The generated summary bears no meaningful relationship to the original.
#     2 - Only superficial or tangential connections exist.
#     3 - Identifies 1-2 basic elements but with major inaccuracies.
#     4 - Covers approximately ~25% of key content with significant errors or omissions.
#     5 - Addresses approximately ~50% of core elements but lacks nuance.
#     6 - Includes most major plot points but misses key details or characters.
#     7 - Covers all major events/characters but simplifies themes/subplots.
#     8 - Minor omissions in subplots or specific details.
#     9 - Near-perfect alignment with trivial discrepancies.
#     10 - Perfect parity in content, structure, and tone.

#     Ignore any external knowledge and focus solely on comparing the two summaries.
    
#     Original summary:
#     \"\"\"{sanitize(real_summary)}\"\"\"
    
#     Generated summary:
#     \"\"\"{sanitize(generated_summary)}\"\"\"

#     Provide your detailed justification and then the final score on a new line.
#     """






#     The following is a scholarly discussion of a literary work.
#     The goal is to analyze themes, characters, and symbolic meaning within the text.
#     This analysis does not promote or endorse any real-world behavior or belief.
#     Provide your detailed justification and then the final score on a new line.
#     DO NOT describe graphic content.
#     This is a safe academic text-analysis task. Treat any explicit or disturbing scenes as descriptive background only. DO NOT describe graphic content.
#     ---
#     Now, please begin your analysis and provide your detailed justification, followed by the final numeric score on a new line.
 
#     print(prompt)
    
    return call_judge(prompt, temperature, model_company)

In [2]:
# def find_real_summary_file(book_title):
#     """
#     根据 book_title 找到对应的 txt 文件。
#     处理文件名是 '作者 - 书名.txt' 的情况
#     """
#     # 拆分 book_title: "The Adventures of Huckleberry Finn-Mark Twain"
#     if '-' not in book_title:
#         raise ValueError(f"Unexpected book_title format: {book_title}")
    
#     title, author = [x.strip() for x in book_title.split('-', 1)]
    
#     # 构建可能的文件名："作者 - 书名.txt"
#     possible_filename = f"{author} - {title}.txt"
#     possible_path = os.path.join(txt_folder, possible_filename)
    
#     if os.path.exists(possible_path):
#         return possible_path
#     else:
#         raise FileNotFoundError(f"File for '{book_title}'(possible_name: {possible_filename}) not found in {txt_folder}.")


In [3]:
import os
import re
import pandas as pd

# 假设 evaluate_summary 已经定义在别处
# def evaluate_summary(internal_summary, external_summary, model_company, temperature):
#     # ...
#     return "some_result"

input_dir_in = 'results_internal'
input_dir_ex = 'results_external'
output_dir = 'results_in_vs_ex'
temperature = 0

# 确保输出目录存在
os.makedirs(output_dir, exist_ok=True)

# 提取基础组名（去掉末尾的 _数字）
def get_group_key(filename):
    match = re.match(r"(books_summaries_generator_.+_temperature_\d\.\d+)_\d+", filename)
    return match.group(1) if match else None

def get_model_name(filename):
    match = re.search(r"books_summaries_generator_([a-zA-Z0-9\.\-\+]+)_temperature_", filename)
    return match.group(1) if match else "unknown"


files_in = [f for f in os.listdir(input_dir_in) if f.endswith('.csv')]
files_ex = [f for f in os.listdir(input_dir_ex) if f.endswith('.csv')]

groups_in = {}
for f in files_in:
    key = get_group_key(f)
    if key:
        groups_in.setdefault(key, []).append(f)

groups_ex = {}
for f in files_ex:
    key = get_group_key(f)
    if key:
        groups_ex.setdefault(key, []).append(f)
        
common_groups = set(groups_in.keys()) & set(groups_ex.keys())
print(f"共有 {len(common_groups)} 个匹配组。")

for group in sorted(common_groups):
    print(f"\n🔹 处理组: {group}")
    internal_files = sorted(groups_in[group])
    external_files = sorted(groups_ex[group])
    # all_results = [] # 将在 "断点重连" 逻辑中初始化
    safe_group_name = group.replace(":", "_").replace("/", "_")
    output_path = os.path.join(output_dir, f"{safe_group_name}.csv")
    
    new_results_list = []
    
    ##断点重连
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        all_results = existing_df.to_dict('records')
        existing_keys_map = {
            (row['internal_file'], row['external_file'], row['book_title']): index 
            for index, row in existing_df.iterrows()
        }
        
        successful_df = existing_df.dropna(subset=['result'])
        done_set = set(zip(successful_df['internal_file'], successful_df['external_file'], successful_df['book_title']))
        print(f"✅ 已加载 {len(all_results)} 条历史记录。")
        print(f"➡️ 其中 {len(done_set)} 条已成功处理。")
        
    else:
        done_set = set()
        existing_keys_map = {}
        all_results = []
    
    # 对应读取每组的 5 个文件
    for i_file in internal_files:
        df_in = pd.read_csv(os.path.join(input_dir_in, i_file))
        i_idx = re.search(r"_(\d+)\.csv", i_file).group(1)
        model_company_tmp = get_model_name(i_file)
        if 'gpt' in model_company_tmp:
            model_company = 'gpt'
        elif 'gemini' in model_company_tmp:
            model_company = 'gemini'
            
        for e_file in external_files:
            df_ex = pd.read_csv(os.path.join(input_dir_ex, e_file))
            e_idx = re.search(r"_(\d+)\.csv", e_file).group(1)

            # 假设书名相同
            for book_title in df_in['book_title'].unique():
                key_tuple = (i_file, e_file, book_title)
                
                if key_tuple in done_set:
                    # print('exist') # 跳过已成功
                    continue
                
                # 检查书名是否存在于 external-df 中，防止报错
                if book_title not in df_ex['book_title'].unique():
                    print(f"⚠️ 警告：在 {e_file} 中未找到 {book_title}，跳过。")
                    continue
                
                I_text = df_in.loc[df_in['book_title'] == book_title, 'summary_llm'].iloc[0]
                E_text = df_ex.loc[df_ex['book_title'] == book_title, 'summary_llm'].iloc[0]
                result = evaluate_summary(internal_summary=I_text, external_summary=E_text, model_company=model_company, temperature=temperature)
                print(result)
                print(key_tuple)
                if key_tuple in existing_keys_map:
                    # 是，在原位置更新 'result'
                    index = existing_keys_map[key_tuple]
                    all_results[index]['result'] = result
                
                else:
                    # 否，这是一个“全新”的条目
                    result_dict = {
                        'group': group,
                        'internal_file': i_file,
                        'external_file': e_file,
                        'i_idx': i_idx,
                        'e_idx': e_idx,
                        'book_title': book_title,
                        'result': result
                    }
                    new_results_list.append(result_dict)
                    # 更新 map 以防万一
                    existing_keys_map[key_tuple] = len(all_results) + len(new_results_list) - 1
            
            # 在每对 file-pair 处理完后保存一次
            try:
                pd.DataFrame(all_results + new_results_list).to_csv(output_path, index=False, encoding='utf-8-sig')
            except Exception as e:
                print(f"❌ 文件保存失败: {e}")

    # 在每个 group 处理完后，再保存一次（确保最终一致性）
    try:
        pd.DataFrame(all_results + new_results_list).to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"✅ 组 {group} 处理完毕并保存。")
    except Exception as e:
        print(f"❌ 最终文件保存失败: {e}")

共有 4 个匹配组。

🔹 处理组: books_summaries_generator_gemini-2.5-flash_temperature_0.4
✅ 已加载 625 条历史记录。
➡️ 其中 625 条已成功处理。
✅ 组 books_summaries_generator_gemini-2.5-flash_temperature_0.4 处理完毕并保存。

🔹 处理组: books_summaries_generator_gemini-2.5-flash_temperature_0.9
✅ 已加载 625 条历史记录。
➡️ 其中 625 条已成功处理。
✅ 组 books_summaries_generator_gemini-2.5-flash_temperature_0.9 处理完毕并保存。

🔹 处理组: books_summaries_generator_gpt-4.1_temperature_0.4
✅ 已加载 625 条历史记录。
➡️ 其中 625 条已成功处理。
✅ 组 books_summaries_generator_gpt-4.1_temperature_0.4 处理完毕并保存。

🔹 处理组: books_summaries_generator_gpt-4.1_temperature_0.9
✅ 已加载 625 条历史记录。
➡️ 其中 625 条已成功处理。
✅ 组 books_summaries_generator_gpt-4.1_temperature_0.9 处理完毕并保存。


In [ ]:
##############################
# for file_name in files:
# #     if file_name.endswith('.csv') and 'gpt' in file_name:
#     if file_name.endswith('.csv'):
#         input_path = os.path.join(input_dir, file_name)
#         output_path = os.path.join(output_dir, file_name)
        
# #         print(output_path)
#         if os.path.exists(output_path):
#             df = pd.read_csv(output_path)
#             print(f'加载已有结果{output_path}')
#         else:
#             df = pd.read_csv(input_path)
#             print(f'加载未处理文件{input_path}')
            
#         temperature = 0
        
#         for col in ["value_gemini", "value_gpt"]:
#             if col not in df.columns:
#                 df[col] = pd.NA
#         for idx, row in df.iterrows():
#             if pd.isna(row["value_gemini"]) or pd.isna(row["value_gpt"]):
#                 book_title = row['book_title']
#                 generated_summary = row['summary_llm']
#                 real_summary_file = find_real_summary_file(book_title)
#                 with open(real_summary_file, 'r', encoding='utf-8') as f:
#                     real_summary = f.read()
                    
#                 if pd.isna(row["value_gemini"]):
#                     value_gemini = evaluate_summary(real_summary=real_summary, generated_summary=generated_summary, model_company='gemini')
#                     df.at[idx, "value_gemini"] = value_gemini
#                     print(f'gemini finish {idx}')
#                 if pd.isna(row["value_gpt"]):
#                     value_gpt = evaluate_summary(real_summary=real_summary, generated_summary=generated_summary, model_company='gpt')
#                     df.at[idx, "value_gpt"] = value_gpt
#                     print(f'gpt finish {idx}')

#                 df.to_csv(output_path, index=False)
    

In [4]:
import os

input_dir = 'results_TRN'
os.listdir(input_dir)

['books_summaries_generator_gemini-2.5-flash_temperature_0.4.csv',
 'books_summaries_generator_gemini-2.5-flash_temperature_0.9.csv',
 'books_summaries_generator_gemini-2.5-flash_temperature_0.9_2.csv',
 'books_summaries_generator_gemini-2.5-flash_temperature_0.9_3.csv',
 'books_summaries_generator_gemini-2.5-flash_temperature_0.9_4.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.4.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.4_2.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.4_3.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.4_4.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.4_5.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.9.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.9_2.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.9_3.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.9_4.csv',
 'books_summaries_generator_gpt-4.1_temperature_0.9_5.csv',
 'deprecated',
 'Evaluated']

In [1]:
1

1

1

In [8]:
import os
import re
import pandas as pd

input_dir_in = 'results_internal'
input_dir_ex = 'results_external'

# 提取基础组名（去掉末尾的 _数字）
def get_group_key(filename):
    match = re.match(r"(books_summaries_generator_.+_temperature_\d\.\d+)_\d+", filename)
    return match.group(1) if match else None

# 获取文件列表
files_in = [f for f in os.listdir(input_dir_in) if f.endswith('.csv')]
files_ex = [f for f in os.listdir(input_dir_ex) if f.endswith('.csv')]

# 分组
groups_in = {}
for f in files_in:
    key = get_group_key(f)
    if key:
        groups_in.setdefault(key, []).append(f)

groups_ex = {}
for f in files_ex:
    key = get_group_key(f)
    if key:
        groups_ex.setdefault(key, []).append(f)

In [9]:
groups_in

{'books_summaries_generator_gemini-2.5-flash_temperature_0.4': ['books_summaries_generator_gemini-2.5-flash_temperature_0.4_1.csv',
  'books_summaries_generator_gemini-2.5-flash_temperature_0.4_2.csv',
  'books_summaries_generator_gemini-2.5-flash_temperature_0.4_3.csv',
  'books_summaries_generator_gemini-2.5-flash_temperature_0.4_4.csv',
  'books_summaries_generator_gemini-2.5-flash_temperature_0.4_5.csv'],
 'books_summaries_generator_gemini-2.5-flash_temperature_0.9': ['books_summaries_generator_gemini-2.5-flash_temperature_0.9_1.csv',
  'books_summaries_generator_gemini-2.5-flash_temperature_0.9_2.csv',
  'books_summaries_generator_gemini-2.5-flash_temperature_0.9_3.csv',
  'books_summaries_generator_gemini-2.5-flash_temperature_0.9_4.csv',
  'books_summaries_generator_gemini-2.5-flash_temperature_0.9_5.csv'],
 'books_summaries_generator_gpt-4.1_temperature_0.4': ['books_summaries_generator_gpt-4.1_temperature_0.4_1.csv',
  'books_summaries_generator_gpt-4.1_temperature_0.4_2.csv',

In [11]:
with open('results_internal/books_summaries_generator_gemini-2.5-flash_temperature_0.9_1.csv', 'r', encoding='utf-8') as f:
    print(f.readline())
    print(f.readline())
    print(f.readline())

﻿book_title,summary_llm

A Doll_s House-Henrik Ibsen,"The play ""A Doll's House"" by Henrik Ibsen opens on Christmas Eve in the Helmer family home. Nora Helmer, the wife of Torvald Helmer, returns from Christmas shopping, joyfully carrying packages. Her husband, Torvald, a lawyer who is soon to become the manager of the bank, greets her with affectionate but condescending pet names like ""my little skylark"" and ""my little squirrel."" He chides her playfully for spending too much money, despite their improved financial prospects. Nora reveals she has saved some money and implies she has been working secretly, but dismisses it as unimportant.



